### 🧹data cleaning and transformation in python to create the dataset to be used analysis

In [27]:
import pandas as pd
import numpy as np


In [28]:
df=pd.read_csv(r"C:\Users\USER\Downloads\LAcity.org\LAcity_Website_Traffic.csv")
df.head()

,Date,Device Category,Browser,# of Visitors,Sessions,Bounce Rate
0,08/27/2019,desktop,Chrome,278,318,50.0
1,08/27/2019,desktop,Internet Explorer,40,40,0.0
2,08/27/2019,mobile,Chrome,199,318,50.0
3,08/27/2019,tablet,Chrome,40,40,100.0
4,08/27/2019,desktop,Firefox,40,40,0.0


### 🔄 Step 1: Standardize Column Names

We'll rename columns to match best practices (snake_case).

In [29]:
df.rename(columns={
    "Date": "date",
    "Device Category": "device_category",
    "Browser": "browser",
    "# of Visitors": "unique_visitors",
    "Sessions": "sessions",
    "Bounce Rate": "bounce_rate"
}, inplace=True)

df.columns


Index(['date', 'device_category', 'browser', 'unique_visitors', 'sessions',
       'bounce_rate'],
      dtype='object')

### 📆 Step 2: Convert the `date` column to datetime format


In [30]:
df['date'] = pd.to_datetime(df['date'], errors='coerce')
df.dtypes


date               datetime64[ns]
device_category            object
browser                    object
unique_visitors             int64
sessions                    int64
bounce_rate               float64
dtype: object

### 🔍 Step 3: converting the date column from full datetime (2019-08-27 00:00:00) to just the date part (2019-08-27)

In [31]:
df['date'] = df['date'].dt.date

### 🔍 Step 4: Explore Device Categories and Browsers


In [32]:
print("Device Categories:", df['device_category'].dropna().unique())
print("\nBrowsers (sorted):", sorted(df['browser'].dropna().unique()))


Device Categories: ['desktop' 'mobile' 'tablet']

Browsers (sorted): ['"mozilla', "'Mozilla", '(not set)', 'AdobeAIR', 'Amazon Silk', 'Android Browser', 'Android Webview', 'BlackBerry', 'BlackBerry9550', 'BlackBerry9630', 'Bluebeam Revu Browser - cef version: 57.0.0.0', 'BrowserNG', 'Camino', 'Chrome', 'Coc Coc', 'DDG-Android-3.0.12', 'Dolfin', 'Easy-Thumb', 'Edge', 'Firefox', 'HggH PhantomJS Screenshoter', 'IE with Chrome Frame', 'Internet Explorer', 'Iron', 'Konqueror', 'MRCHROME', 'Maxthon', 'Mozilla', 'Mozilla Compatible Agent', 'NetFront', 'Netscape', 'Nintendo 3DS Browser', 'Nintendo Browser', 'Nokia Browser', 'Opera', 'Opera Mini', 'Peeplo Screenshot Bot', 'Phantom Browser', 'Phantom.js bot', 'PlayFreeBrowser', 'Playstation 3', 'Playstation 4', 'Playstation Vita Browser', 'Puffin', 'S40 Ovi Browser', 'Safari', 'Safari (in-app)', 'Samsung Internet', 'SeaMonkey', 'UC Browser', 'UCWEB', 'WeSEE_Bot:we_help_monitize_your_site', 'YE', 'YaBrowser', 'YelpWebView', 'fuelbot', 'mozilla', 

### 🧹 Step 5: Clean Formatting Issues
- Strip whitespace
- Remove quotation marks


In [33]:

df['browser'] = df['browser'].str.strip().str.strip('"').str.strip("'").str.title()

### ✏️ Step 6: Standardize Some Browser Values (if needed)


In [34]:
# Define browser normalization map
browser_mapping = {
    # Mozilla-related
    "Mozilla Compatible Agent": "Mozilla",
    "Mozilla": "Mozilla",
    "Mozilla": "Mozilla",

    # Firefox
    "Firefox": "Firefox",
    "Mozilla": "Mozilla",  # Optional merge if you decide to combine

    # Internet Explorer
    "Ie With Chrome Frame": "Internet Explorer",
    "Internet Explorer": "Internet Explorer",

    # Playstation
    "Playstation 3": "Playstation Browser",
    "Playstation 4": "Playstation Browser",
    "Playstation Vita Browser": "Playstation Browser",

    # Bots
    "Phantom.Js Bot": "Bot",
    "Hggh Phantomjs Screenshoter": "Bot",
    "Peeplo Screenshot Bot": "Bot",
    "Wesee_Bot:We_Help_Monitize_Your_Site": "Bot",
    "Pr-Cy.Ru Screenshot Bot": "Bot",
    "Fuelbot": "Bot",

    # Other (keep as-is or define new mappings later)
}

# Apply mapping (map returns NaN for unmapped values, so fillna with original value)
df['browser'] = df['browser'].map(browser_mapping).fillna(df['browser'])

# Handle placeholder or missing browser values
df['browser'] = df['browser'].replace('(Not Set)', pd.NA)

# Review cleaned browser values
print("\nCleaned browser values:")
print(df['browser'].dropna().unique())


Cleaned browser values:
['Chrome' 'Internet Explorer' 'Firefox' 'Edge' 'Safari' 'Amazon Silk'
 'Mozilla' 'Opera Mini' 'Samsung Internet' 'Safari (In-App)'
 'Android Webview' 'Opera' 'Android Browser' 'Yabrowser' 'Nokia Browser'
 'Playstation Browser' 'Blackberry' 'Coc Coc' 'Puffin' 'Uc Browser'
 'Mrchrome' 'Seamonkey' 'Bot'
 'Bluebeam Revu Browser - Cef Version: 57.0.0.0' 'Maxthon' 'Ucweb'
 'Adobeair' 'Iron' 'Yelpwebview' 'Easy-Thumb' 'Nintendo Browser'
 'Netfront' 'Ddg-Android-3.0.12' 'Ye' 'Nintendo 3Ds Browser'
 'Blackberry9630' 'S40 Ovi Browser' 'Samsung-Gt-I9500' 'Blackberry9550'
 'Konqueror' 'Dolfin' 'Phantom Browser' 'Camino' 'Netscape'
 'Playfreebrowser' 'Browserng']


### ❓ Step 7: Replace '(not set)' with Nulls
- Handling Null Browser Values

In [35]:
df['browser'] = df['browser'].replace('(not set)', np.nan)


In [36]:
df['browser'] = df['browser'].fillna('Unknown')

### 🔍 Step 8: Count Missing Values Across Key Columns


In [37]:
null_counts = df[['date', 'device_category', 'unique_visitors', 'sessions', 'bounce_rate']].isnull().sum()
null_counts


date               0
device_category    0
unique_visitors    0
sessions           0
bounce_rate        0
dtype: int64

### 📊 Step 9: Summary Statistics


In [38]:
summary = {
    "start_date": df['date'].min(),
    "end_date": df['date'].max(),
    "device_count": df['device_category'].nunique(),
    "browser_count": df['browser'].nunique(),
    "min_visitors": df['unique_visitors'].min(),
    "max_visitors": df['unique_visitors'].max(),
    "min_sessions": df['sessions'].min(),
    "max_sessions": df['sessions'].max(),
    "min_bounce": df['bounce_rate'].min(),
    "max_bounce": df['bounce_rate'].max()
}
summary


{'start_date': datetime.date(2014, 1, 1),
 'end_date': datetime.date(2019, 8, 27),
 'device_count': 3,
 'browser_count': 47,
 'min_visitors': 35,
 'max_visitors': 34365,
 'min_sessions': 35,
 'max_sessions': 46172,
 'min_bounce': 0.0,
 'max_bounce': 100.0}

### 🔍 Step 10 duplicates check 

In [39]:
# Define the subset of columns to check for duplicates
duplicate_columns = ['date', 'device_category', 'browser', 'unique_visitors', 'sessions', 'bounce_rate']

# Find all duplicates across these columns (excluding the first occurrence)
duplicates = df[df.duplicated(subset=duplicate_columns)]

print(f"Duplicate rows (excluding first): {len(duplicates)}")


Duplicate rows (excluding first): 5653086


In [40]:
duplicates_all = df[df.duplicated(subset=duplicate_columns, keep=False)]

print(f"All duplicate rows: {len(duplicates_all)}")


All duplicate rows: 6675817


In [41]:
df = df.drop_duplicates(subset=duplicate_columns, keep='first')


In [42]:
print(f"Total duplicate rows: {df.duplicated(subset=['date', 'device_category', 'browser', 'unique_visitors', 'sessions', 'bounce_rate']).sum()}")


Total duplicate rows: 0


### Ready for further exploration and visualization

### 💾 Step 11:  Save Cleaned Data


In [43]:
df.to_csv("cleaned_traffic.csv", index=False)


In [44]:
df.head()

,date,device_category,browser,unique_visitors,sessions,bounce_rate
0,2019-08-27,desktop,Chrome,278,318,50.0
1,2019-08-27,desktop,Internet Explorer,40,40,0.0
2,2019-08-27,mobile,Chrome,199,318,50.0
3,2019-08-27,tablet,Chrome,40,40,100.0
4,2019-08-27,desktop,Firefox,40,40,0.0


In [45]:
print(df.describe())


       unique_visitors      sessions   bounce_rate
count     2.695899e+06  2.695899e+06  2.695899e+06
mean      1.394225e+03  1.574463e+03  4.905789e+01
std       1.282043e+03  1.469549e+03  1.815124e+01
min       3.500000e+01  3.500000e+01  0.000000e+00
25%       5.950000e+02  6.650000e+02  3.772000e+01
50%       1.063000e+03  1.189000e+03  4.641000e+01
75%       1.825000e+03  2.044000e+03  5.758000e+01
max       3.436500e+04  4.617200e+04  1.000000e+02
